In [128]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

In [129]:
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
mind=temp[temp['Fav_Odds']<1.3].copy()
mind=mind[mind['Fav_Odds']>1]
mind=mind[mind['Date']>='2024-01-01']

favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
mind=mind[(mind['Fav']).isin(fav_list)]
mind=mind[mind['dog_rank']>50]
len(mind[mind['Fav']==mind['Winner']])/len(mind),len(mind)
mind[['Fav','Dog','Fav_Odds','Winner','Fav_Win','dog_rank']].to_clipboard(index=False)



In [130]:
import pandas as pd

# Assuming 'temp' is your original dataframe
calc = temp.copy()

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets (i.e., wins for Fav and Dog)
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Sort player results by Player and Date
player_results = player_results.sort_values(['Player', 'Date'])

# Create a dictionary to store the wins for each player
player_win_counts = {}

# Function to calculate rolling wins in the last 30 days for each player
def count_rolling_wins(player, match_date):
    if player not in player_win_counts:
        player_win_counts[player] = []
    
    # Filter player results within the last 30 days from the current match date
    recent_wins = player_results[(player_results['Player'] == player) & 
                                 (player_results['Date'] <= match_date) & 
                                 (player_results['Date'] > match_date - pd.Timedelta(days=30))]
    
    # Count the number of wins in this 30-day window
    return recent_wins['Win'].sum()

# Create new columns for the calculated rolling win counts for Fav and Dog
fav_wins_last_30 = []
dog_wins_last_30 = []

# Loop through each match in the original dataframe 'calc' to calculate the rolling wins for both Fav and Dog
for _, row in calc.iterrows():
    fav_wins = count_rolling_wins(row['Fav'], row['Date'])
    dog_wins = count_rolling_wins(row['Dog'], row['Date'])
    
    fav_wins_last_30.append(fav_wins)
    dog_wins_last_30.append(dog_wins)

# Add these calculated win counts as new columns in the 'calc' dataframe
calc['Fav_Wins_Last_30_Days_Before'] = fav_wins_last_30
calc['Dog_Wins_Last_30_Days_Before'] = dog_wins_last_30

# Ensure any missing counts are filled with 0 (in case the player has no matches in the last 30 days)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


In [131]:
twst=calc[(calc['Fav'] == 'Rei Sakamoto') | (calc['Dog'] == 'Rei Sakamoto')]
twst=calc[(calc['Fav'] == 'Thiago Agustin Tirante') | (calc['Dog'] == 'Thiago Agustin Tirante')]
twst[twst['Date']>'2025-05-01'][['Date','Winner','Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before']]

,Date,Winner,Fav,Dog,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before
6760,2025-05-02,Roman Andres Burruchaga,Thiago Agustin Tirante,Roman Andres Burruchaga,2,3
7241,2025-05-14,Thiago Agustin Tirante,Thiago Agustin Tirante,Ryan Nijboer,3,1
7297,2025-05-15,Hady Habib,Thiago Agustin Tirante,Hady Habib,4,7
7304,2025-05-15,Thiago Agustin Tirante,Thiago Agustin Tirante,Alberto Olivieri Genaro,4,3
7433,2025-05-19,Thiago Agustin Tirante,Thiago Agustin Tirante,Juan Pablo Ficovich,5,0
7539,2025-05-21,Thiago Agustin Tirante,Thiago Agustin Tirante,August Holmgren,6,0
7627,2025-05-22,Ethan Quinn,Ethan Quinn,Thiago Agustin Tirante,5,6
7694,2025-05-25,Damir Dzumhur,Damir Dzumhur,Thiago Agustin Tirante,2,6
8288,2025-06-17,Thiago Agustin Tirante,Thiago Agustin Tirante,Denis Yevseyev,3,1
8316,2025-06-18,Thiago Agustin Tirante,Thiago Agustin Tirante,Viktor Durasovic,3,4


In [132]:
calc['Predicted_Winner'] = calc['Fav_Wins_Last_30_Days_Before'] < calc['Dog_Wins_Last_30_Days_Before']
accuracy = (calc['Predicted_Winner'] == calc['Dog_Win']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 74.70%


In [133]:
stake=100
date_filter=calc[calc['Date']>'2024-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']+1]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_ClayResults_Womens.csv')

Total Profit: 15440.0 174 0.7413793103448276


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
2470,2024-06-02,3.07,Mirra Andreeva,Peyton Stearns,Mirra Andreeva,4,2,7,2
2482,2024-06-03,2.34,Ons Jabeur,Clara Tauson,Ons Jabeur,2,2,5,3
2512,2024-06-04,3.91,Mirra Andreeva,Varvara Gracheva,Mirra Andreeva,7,2,6,3
2543,2024-06-05,2.38,Noma Noha Akugue,Noma Noha Akugue,Polina Kudermetova,2,1,2,0
2638,2024-06-08,1.89,Tamara Zidansek,Tamara Zidansek,Martina Trevisan,3,3,5,2
...,...,...,...,...,...,...,...,...,...
8223,2025-06-14,3.68,Barbora Palicova,Barbora Palicova,Julia Grabher,2,2,3,1
8589,2025-07-08,2.06,Mona Barthel,Mona Barthel,Tamara Korpatsch,6,4,3,1
8595,2025-07-08,2.23,Louisa Chirico,Louisa Chirico,Moyuka Uchijima,8,4,3,0
8612,2025-07-09,2.68,Ekaterina Kazionova,Ekaterina Kazionova,Miriam Bulgaru,2,3,2,0


In [134]:
stake=100
date_filter=calc[calc['Date']>'2025-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']+1<oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] == row['Fav'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_ClayResults_Womens.csv')

Total Profit: 206.0 7 0.7142857142857143


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
7981,2025-06-03,1.89,Ana Bogdan,Ana Bogdan,Varvara Lepchenko,3,3,1,4
8042,2025-06-06,2.06,Aryna Sabalenka,Iga Swiatek,Aryna Sabalenka,0,4,6,9
8099,2025-06-10,1.98,Petra Marcinko,Panna Udvardy,Petra Marcinko,8,1,0,2
8171,2025-06-11,1.88,Petra Marcinko,Tatiana Prozorova,Petra Marcinko,0,1,1,3
8584,2025-07-08,2.04,Katarzyna Kawa,Katarzyna Kawa,Nuria Parrizas Diaz,9,7,1,3
8598,2025-07-09,1.97,Louisa Chirico,Jana Fett,Louisa Chirico,4,2,0,4
8653,2025-07-10,1.89,Petra Marcinko,Victoria Jimenez Kasintseva,Petra Marcinko,7,1,1,3


In [135]:
stake=100
date_filter=calc[calc['Date']>'2025-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']+1]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_ClayResults_Mens.csv')

Total Profit: 8440.0 105 0.6952380952380952


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
7937,2025-06-02,5.79,Vitaliy Sachko,Vitaliy Sachko,Jakub Mensik,6,3
7947,2025-06-02,2.32,Milos Karol,Milos Karol,Andrej Martin,4,1
7954,2025-06-03,3.20,Alejandro Tabilo,Hynek Barton,Alejandro Tabilo,4,1
7956,2025-06-03,2.48,Milos Karol,Milos Karol,Jozef Kovalik,5,0
7959,2025-06-03,8.49,Alexander Bublik,Alexander Bublik,Jack Draper,10,6
...,...,...,...,...,...,...,...
8675,2025-07-12,2.31,Mariano Navone,Filip Misolic,Mariano Navone,8,4
8677,2025-07-12,2.95,Facundo Diaz Acosta,Stefano Travaglia,Facundo Diaz Acosta,9,4
8680,2025-07-12,2.51,Elmer Moller,Francesco Maestrelli,Elmer Moller,9,3
8682,2025-07-12,2.33,Gonzalo Bueno,Gonzalo Bueno,Marc Andrea Huesler,4,0


In [136]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']+1<oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] == row['Fav'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_ClayResults_Mens.csv')

Total Profit: 6154.0 136 0.8161764705882353


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
4940,2025-01-13,1.88,Mariano Kestelboim,Alex Barrena,Mariano Kestelboim,4,0,0,2
4944,2025-01-13,1.97,Mariano Kestelboim,Lorenzo Lingua Lavallen Alejo,Mariano Kestelboim,1,0,0,2
4971,2025-01-17,2.01,Daniel Vallejo Adolfo,Alberto Olivieri Genaro,Daniel Vallejo Adolfo,11,4,1,3
5002,2025-01-22,1.87,Gonzalo Bueno,Albert Ramos Vinolas,Gonzalo Bueno,8,5,0,3
5004,2025-01-23,1.88,Gonzalo Bueno,Murkel Dellien,Gonzalo Bueno,7,5,2,4
...,...,...,...,...,...,...,...,...,...
8514,2025-07-06,1.94,Gerard Campana Lee,Gerard Campana Lee,Federico Bondioli,10,2,2,6
8517,2025-07-06,2.02,Jan Choinski,Calvin Hemery,Jan Choinski,12,6,3,5
8524,2025-07-06,1.92,Alessandro Pecci,Alessandro Pecci,Stefanos Sakellaridis,3,1,1,4
8658,2025-07-11,1.91,Sumit Nagal,Sumit Nagal,Marco Cecchinato,9,15,5,10


In [137]:
from sqlalchemy import text

#with devengine.connect() as conn:
#    conn.execute(text('DELETE FROM results_clay_1 WHERE Date > "2025-06-07"'))
#    conn.commit()
#pd.read_sql_query('Select distinct* from AllMatches where date = "2025-06-08" and Sex like "Womens"',con=devengine)

In [138]:
tempo = temp.copy()
date_filter = pd.read_pickle(r'.\Clay_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
date_filter['Date'] = date_string
calc=pd.concat([tempo, date_filter], ignore_index=True)


In [139]:
import pandas as pd


# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets (i.e., wins for Fav and Dog)
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Sort player results by Player and Date
player_results = player_results.sort_values(['Player', 'Date'])

# Create a dictionary to store the wins for each player
player_win_counts = {}

# Function to calculate rolling wins in the last 30 days for each player
def count_rolling_wins(player, match_date):
    if player not in player_win_counts:
        player_win_counts[player] = []
    
    # Filter player results within the last 30 days from the current match date
    recent_wins = player_results[(player_results['Player'] == player) & 
                                 (player_results['Date'] <= match_date) & 
                                 (player_results['Date'] > match_date - pd.Timedelta(days=30))]
    
    # Count the number of wins in this 30-day window
    return recent_wins['Win'].sum()

# Create new columns for the calculated rolling win counts for Fav and Dog
fav_wins_last_30 = []
dog_wins_last_30 = []

# Loop through each match in the original dataframe 'calc' to calculate the rolling wins for both Fav and Dog
for _, row in calc.iterrows():
    fav_wins = count_rolling_wins(row['Fav'], row['Date'])
    dog_wins = count_rolling_wins(row['Dog'], row['Date'])
    
    fav_wins_last_30.append(fav_wins)
    dog_wins_last_30.append(dog_wins)

# Add these calculated win counts as new columns in the 'calc' dataframe
calc['Fav_Wins_Last_30_Days_Before'] = fav_wins_last_30
calc['Dog_Wins_Last_30_Days_Before'] = dog_wins_last_30

# Ensure any missing counts are filled with 0 (in case the player has no matches in the last 30 days)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


In [150]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1]
today=today[today['Dog_Wins_Last_30_Days_Before']>today['Fav_Wins_Last_30_Days_Before']+2]
womens=today[today['Sex']=='Womens']
womens[['Time','Dog','Dog_Odds','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]


,Time,Dog,Dog_Odds,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before


In [148]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1]
today=today[today['Dog_Wins_Last_30_Days_Before']>today['Fav_Wins_Last_30_Days_Before']+2]
mens=today[today['Sex']!='Womens']
mens[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
8722,01:30,Sumit Nagal,1.97,Timofey Skatov,1.78,5,1
8726,03:10,Titouan Droguet,2.08,Elmer Moller,1.69,7,4
8731,19:30,Dalibor Svrcina,2.02,Roman Andres Burruchaga,1.78,4,0


In [149]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_Before']+2<today['Fav_Wins_Last_30_Days_Before']]
mens_favs=today[today['Sex']!='Womens']
mens_favs[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
8752,23:00,Jelle Sels,1.91,Federico Bondioli,1.78,0,3


In [143]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_Before']+1<today['Fav_Wins_Last_30_Days_Before']]
womens_favs=today[today['Sex']=='Womens']
womens_favs[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before


In [144]:
womens_favs[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('clay_womens_favs',con=devengine,if_exists='append')
mens_favs[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('clay_mens_favs',con=devengine,if_exists='append')
mens[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('clay_mens',con=devengine,if_exists='append')
womens[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('clay_womens',con=devengine,if_exists='append')

1